In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_iris
from sklearn.decomposition import TruncatedSVD, PCA

In [37]:
X = pd.DataFrame([
    [1,0,0,0],
    [0,0,0,4],
    [0,3,0,0],
    [0,0,0,0],
    [2,0,0,0],
])
X

,0,1,2,3
0,1,0,0,0
1,0,0,0,4
2,0,3,0,0
3,0,0,0,0
4,2,0,0,0


In [52]:
# 对A进行截断奇异值分解
svd = TruncatedSVD(n_components=2, random_state=2)
X_transform = svd.fit_transform(X)
np.around(X_transform, 7)

array([[0., 0.],
       [4., 0.],
       [0., 3.],
       [0., 0.],
       [0., 0.]])

V<sup>T</sup> 的行向量是X<sup>T</sup>X的单位特征向量组成

In [51]:
VT = np.around(svd.components_, 7) # V.T的值（四舍五入保留7位小数）
VT

array([[-0.,  0.,  0.,  1.],
       [ 0.,  1., -0., -0.]])

In [82]:
np.dot(VT, X.T) #经过正交变换得到新矩阵

array([[0., 4., 0., 0., 0.],
       [0., 0., 3., 0., 0.]])

查看降维结果

In [4]:
X = pd.DataFrame([
    [1,2. ,0,0],
    [0,0. ,0,4],
    [0,0., 0.1,0],
    [0,0., 0,0],
    [2,4.1,0,0],
])
X

,0,1,2,3
0,1,2.0,0.0,0
1,0,0.0,0.0,4
2,0,0.0,0.1,0
3,0,0.0,0.0,0
4,2,4.1,0.0,0


In [5]:
# 对A进行截断奇异值分解
svd = TruncatedSVD(n_components=3, random_state=2)
X_transform = svd.fit_transform(X)
np.around(X_transform, 7)

array([[ 2.2359981,  0.       , -0.       ],
       [-0.       ,  4.       ,  0.       ],
       [ 0.       , -0.       ,  0.1      ],
       [ 0.       ,  0.       ,  0.       ],
       [ 4.5617897,  0.       , -0.       ]])

In [12]:
np.around(svd.explained_variance_, 5)

array([3.31353e+00, 2.56000e+00, 1.60000e-03])

第一列v1和第二列v2的相关性非常强，且var(v1)<var(v2) 所以奇异值转换后v1对应的v1<sup>'</sup>是最小的主成分，被截断了

## 应用

### 在PCA降维
在主成分分析（PCA）原理总结中，我们讲到要用PCA降维，需要找到样本协方差矩阵XTX
的最大的d个特征向量，然后用这最大的d个特征向量张成的矩阵来做低维投影降维。可以看出，在这个过程中需要先求出协方差矩阵XTX
，当样本数多样本特征数也多的时候，这个计算量是很大的。

　　　　注意到我们的SVD也可以得到协方差矩阵XTX
最大的d个特征向量张成的矩阵，但是SVD有个好处，有一些SVD的实现算法可以不求先求出协方差矩阵XTX
，也能求出我们的右奇异矩阵V
。也就是说，我们的PCA算法可以不用做特征分解，而是做SVD来完成。这个方法在样本量很大的时候很有效。实际上，scikit-learn的PCA算法的背后真正的实现就是用的SVD，而不是我们我们认为的暴力特征分解。

<strong>SVD和PCA的区别</strong>:
 SVD算法相较于使用SVD的PCA,PCA需要对样本中心化,而SVD不需要中心化

In [76]:
# PCA对X降维
X_dr = PCA(2).fit_transform(X)
np.around(X_dr, 2)

array([[-0.64, -0.86],
       [ 3.29,  0.36],
       [-1.36,  2.21],
       [-0.48, -0.4 ],
       [-0.81, -1.31]])

用SVD按PCA的思路实现X的降维

In [80]:
# 中心化
x_means = X.mean(axis=0)
print(x_means)
def center(row):
    return row-x_means
X_center = X.apply(center,axis=1)
print(X_center)
# 奇异值分解
svd2 = TruncatedSVD(n_components=2, random_state=2)
X_center_transform = svd2.fit_transform(X_center)
np.around(X_center_transform, 2)

0    0.6
1    0.6
2    0.0
3    0.8
dtype: float64
     0    1    2    3
0  0.4 -0.6  0.0 -0.8
1 -0.6 -0.6  0.0  3.2
2 -0.6  2.4  0.0 -0.8
3 -0.6 -0.6  0.0 -0.8
4  1.4 -0.6  0.0 -0.8


array([[-0.64, -0.86],
       [ 3.29,  0.36],
       [-1.36,  2.21],
       [-0.48, -0.4 ],
       [-0.81, -1.31]])

### 数据压缩

### 降噪